In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
from pathlib import Path
import torch
import torch.nn.functional as F
import torchaudio
from matplotlib import pyplot as plt
from IPython.display import Audio, display
from tqdm import tqdm

p = Path('../datasets') 
df = pd.read_csv(p / 'AUMILAB/metadata/metadata.txt', sep=' ')

In [2]:
# Para crear AUMILAB10s

(p / 'AUMILAB10s').mkdir(exist_ok=True)
(p / 'AUMILAB10s/audios').mkdir(exist_ok=True)
(p / 'AUMILAB10s/metadata').mkdir(exist_ok=True)

pd.set_option('mode.chained_assignment', None)
seconds_per_block = 10
new_csv = []

for file, metadata in tqdm(df.groupby('filename')):
    waveform, Fs = torchaudio.load(p / f'AUMILAB/audios/{file}')
    splitted_waveforms = torch.split(waveform, Fs*seconds_per_block, dim=-1)
    splitted_metadata = []
    for k, new_waveform in enumerate(splitted_waveforms):
        if new_waveform.shape[-1] < Fs*seconds_per_block: # Force 10 seconds
            pad_length = Fs*10 - new_waveform.shape[-1]
            new_waveform = F.pad(input=new_waveform, pad=(0, pad_length), mode='constant', value=0)
        torchaudio.save(p / f'AUMILAB10s/audios/{file.split(".")[0]}_{k}.wav', new_waveform, Fs)

        mask = (metadata['start'] < seconds_per_block*(k+1)) & (metadata['end'] > seconds_per_block*k)
        new_metadata = metadata.loc[mask]
        new_metadata[['start', 'end']] = new_metadata[['start',  'end']].transform(lambda x: x - seconds_per_block*k)
        new_metadata.loc[new_metadata['start'] < 0., 'start'] = 0.
        new_metadata.loc[new_metadata['end'] > 10., 'end'] = 10.
        new_metadata.loc[:, "filename"] = new_metadata["filename"].apply(lambda name: f'{name.split(".")[0]}_{k}.wav')
        splitted_metadata.append(new_metadata)
    new_csv.append(pd.concat(splitted_metadata, ignore_index=True))
    #break # SACALE EL FRENO y DEJALO CORRER
pd.concat(new_csv, ignore_index=True).to_csv(p / 'AUMILAB10s/metadata/metadata.txt', index=False, sep=" ")
    

100%|█████████| 6032/6032 [05:42<00:00, 17.62it/s]


In [ ]:
!head '../datasets/AUMILAB10s/metadata/metadata.txt'

from fusanet_utils.datasets.aumilab import AUMILAB

dataset = AUMILAB('..')
len(dataset)

In [ ]:
waveform.shape

In [ ]:

result.shape

In [ ]:
filename, metadata = dataset[14802]
display(filename,  metadata)
waveform, Fs = torchaudio.load(filename)

fig, ax = plt.subplots()
ax.plot(waveform[0, :])
Audio(waveform, rate=Fs)

In [ ]:
df.loc[ df["filename"].apply(lambda name: filename.stem[:-2] in name)]